In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from datetime import datetime as dt
import warnings
 
warnings.filterwarnings("ignore")
plt.rc('font', family='NanumBarunGothic') 

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **1.신한카드 전체에 대한 타격, 회복 그래프**

In [ ]:
scr = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/CARD_SPENDING_RESIDENT.txt', sep='\t', encoding='cp949')

dong = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/지역데이터/전체동별클러스터링.csv', sep = ',', encoding='utf-8')
dong_cd = pd.read_csv('./gdrive/My Drive/빅콘 대상팀/data/SC/SC동코드변환.csv', sep = ',', encoding='utf-8')

In [ ]:
# 신한카드 전처리

# 코드 -> 이름으로 바꾸는 함수 : name_a_to_b
def name_a_to_b(data,col,a,b):
  data = data.replace({col:a}, {col:b}, inplace=True)
  return(data)

# 카테고리 코드 -> 이름
cat_code = [10,20,21,22,30,31,32,33,34,35,40,42,43,44,50,52,60,62,70,71,80,81,92]
cat_name = ['숙박','레저용품','레저업소','문화취미','가구','전기','주방용구',
            '연료판매','광학제품','가전','유통업','의복','직물','신변잡화','서적문구','사무통신',
            '자동차판매','자동차정비','의료기관','보건위생','요식업소','음료식품','수리서비스']
for i in range(len(cat_code)):
  name_a_to_b(scr,'MCT_CAT_CD',cat_code[i],cat_name[i])

# 시구 및 동 이름 추가
scr = pd.merge(scr, dong_cd, left_on=['GU_CD', 'DONG_CD'], right_on=['GU_CD', 'HDONG_CD'])

# 날짜 형식으로 변환(int->str->date)
scr['STD_YMD'] = pd.to_datetime(scr['STD_DD'].astype(str), format='%Y%m%d',infer_datetime_format=True)

# 클러스터링 추가
scr = pd.merge(scr, dong, left_on='HDONG_NM', right_on='HDONG_NM')

# 연도 추가
scr['STD_Y'] = scr['STD_DD'].astype(str).str.slice(0,4)

scr = scr.drop(['STD_DD', 'GU_CD', 'DONG_CD', 'HDONG_CD', 'AGE_CD'], axis=1)

In [ ]:
scr_s = scr.query("SIDO_NM == '서울'")

In [ ]:
def growth(data):
  sc_cnt_sum_19 =data[data['STD_YMD'] < '2020']
  sc_cnt_sum_20 =data[data['STD_YMD'] > '2020']

  sc_cnt_sum_20.index = sc_cnt_sum_20['STD_YMD']
  del sc_cnt_sum_20['STD_YMD']
  sc_cnt_sum_19.index = sc_cnt_sum_19['STD_YMD']
  del sc_cnt_sum_19['STD_YMD']

  #2020 index 저장
  index_2020 = list(sc_cnt_sum_20.index)

  # rolling 전 요일을 맞춰주기 위한 작업

  #20/02/01 = 일요일
  #19/02/01 = 토요일 ==> 19년 토요일 데이터 삭제
  sc_cnt_sum_19=sc_cnt_sum_19.iloc[1:,]

  #설날 보정
  columns = sc_cnt_sum_19.columns
  index = sc_cnt_sum_19.index

  sc_cnt_sum_19 = np.array(sc_cnt_sum_19)
  sc_cnt_sum_19[2:5,:] = (sc_cnt_sum_19[9:12,:]+sc_cnt_sum_19[16:19,:])/2
  sc_cnt_sum_19 = pd.DataFrame(sc_cnt_sum_19)
  sc_cnt_sum_19.columns = columns
  sc_cnt_sum_19.index = index

  sc_cnt_sum_19_roll7 = pd.DataFrame(sc_cnt_sum_19.rolling(7,min_periods = 1).mean()).reset_index()
  sc_cnt_sum_20_roll7 = pd.DataFrame(sc_cnt_sum_20.rolling(7,min_periods = 1).mean()).reset_index()

  del sc_cnt_sum_19_roll7['STD_YMD']
  del sc_cnt_sum_20_roll7['STD_YMD']

  sc_cnt_sum_20_roll7 = sc_cnt_sum_20_roll7.iloc[:-2,:]

  sc_cnt_sum_per= (sc_cnt_sum_20_roll7-sc_cnt_sum_19_roll7)/sc_cnt_sum_19_roll7

  index_2020=[x.strftime('%m-%d')for x in index_2020]
  sc_cnt_sum_per.index = index_2020[:-2]

  return sc_cnt_sum_per

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

def slope(data):
  cat = data.columns

  r_sq=[]
  slope=[]
  intercept=[]
  for c in cat:
    x = np.arange(0,data.shape[0]/100,0.01).reshape(data.shape[0],-1)
    y = np.array(data.loc[:,c]).reshape(-1,)
    model = LinearRegression().fit(x, y)
    r_sq.append(model.score(x, y))
    intercept.append(model.intercept_)
    slope.append(model.coef_)
  
  return r_sq, intercept, slope

In [ ]:
def draw_slope(data):
  sc_1 =  data.loc[data.index < '02-18',:]
  sc_2 =  data.loc[(data.index >= '02-18') & (data.index < '03-22'),:]
  sc_3 =  data.loc[data.index >= '03-22',:]

  before_r, before_intercept, before_slope = slope(sc_1)
  middle_r, middle_intercept, middle_slope = slope(sc_2)
  after_r, after_intercept, after_slope = slope(sc_3)

  before = pd.DataFrame(
      {
      'slope_before' : np.array(before_slope).reshape(-1,),
      'inter_before' : before_intercept,
      'r2_before': before_r,
  }).T
  before.columns = sc_1.columns

  middle = pd.DataFrame(
      {
      'slope_middle' : - np.array(middle_slope).reshape(-1,),
      'inter_middle' : middle_intercept,
      'r2_middle': middle_r,
  }).T
  middle.columns = sc_2.columns

  after = pd.DataFrame({
      'slope_after': np.array(after_slope).reshape(-1,),
      'inter_after' : after_intercept,
      'r2_after':after_r    
  }).T   
  after.columns = sc_3.columns

  merge = pd.concat([middle.iloc[0,:],after.iloc[0,:]],axis=1)  
  merge['cat'] = merge.index
  merge['cat'] = merge['cat'].apply(lambda x : x[7:])

  return merge

In [ ]:
def show_scattor(data, p1, p2, name):
    fig = px.scatter(data, x="slope_middle", y="slope_after", hover_data=["cat"], text="cat",template= 'plotly_white') # ,color = 'group'
    fig.update_traces(textfont_size=7)

    fig.update_traces(textposition='top center')

    fig.update_layout(
        showlegend=False,
        title="회복도 및 타격도 / 성장률 & "+name,
        xaxis_title="타격도 (신천지 - 사회적 거리두기)",
        yaxis_title="회복도 (사회적 거리두기 - 최근)",
        autosize=False,
        width=600,
        height=600,
        margin=dict(l=50,r=50,b=100,t=100,pad=4))


    fig.add_shape(dict(type="line",
                x0=p1,y0=p1,
                x1=p2,y1=p2,
                line=dict(color="red",width=1,dash="dash",)))

    fig.show()

In [ ]:
def rename_col(X):
    X.columns =['SC_CNT_가구', 'SC_CNT_가전',
       'SC_CNT_광학제품', 'SC_CNT_레저업소', 'SC_CNT_레저용품',
       'SC_CNT_문화취미', 'SC_CNT_보건위생', 'SC_CNT_사무통신',
       'SC_CNT_서적문구', 'SC_CNT_수리서비스', 'SC_CNT_숙박',
       'SC_CNT_신변잡화', 'SC_CNT_연료판매', 'SC_CNT_요식업소',
       'SC_CNT_유통업', 'SC_CNT_음료식품', 'SC_CNT_의료기관',
       'SC_CNT_의복', 'SC_CNT_자동차정비', 'SC_CNT_자동차판매',
       'SC_CNT_전기', 'SC_CNT_주방용구', 'SC_CNT_직물',
       'SC_AMT_가구', 'SC_AMT_가전', 'SC_AMT_광학제품',
       'SC_AMT_레저업소', 'SC_AMT_레저용품', 'SC_AMT_문화취미',
       'SC_AMT_보건위생', 'SC_AMT_사무통신', 'SC_AMT_서적문구',
       'SC_AMT_수리서비스', 'SC_AMT_숙박', 'SC_AMT_신변잡화',
       'SC_AMT_연료판매', 'SC_AMT_요식업소', 'SC_AMT_유통업',
       'SC_AMT_음료식품', 'SC_AMT_의료기관', 'SC_AMT_의복',
       'SC_AMT_자동차정비', 'SC_AMT_자동차판매', 'SC_AMT_전기',
       'SC_AMT_주방용구', 'SC_AMT_직물']
    return X

In [ ]:
scr_s = scr_s.groupby(['STD_YMD', 'MCT_CAT_CD']).sum().unstack(level=1).fillna(0)
scr_s = rename_col(scr_s)
scr_s = scr_s.iloc[:,:23]
scr_s = scr_s.reset_index(drop=False)
scr_s = growth(scr_s)
xx = scr_s
scr_s = draw_slope(scr_s)

In [ ]:
group1 = scr_s.query('cat in ["서적문구" , "숙박", "문화취미", "유통업", "사무통신", "전기","레저업소"]')
group2 = scr_s.query('cat in ["요식업소", "보건위생", "음료식품", "연료판매", "광학제품", "수리서비스", "의료기관"]')
group3 = scr_s.query('cat in ["자동차정비", "레저용품", "의복", "신변잡화", "자동차판매", "주방용구", "직물","가구","가전"]')

In [ ]:
scr_s['group'] = scr_s['cat'].apply(lambda x : 'CAT3' if x in list(group1.cat) else ('CAT2' if x in list(group2.cat) else 'CAT1'))

In [ ]:
scr_s['catnm'] = scr_s['cat'].apply(lambda x: '' if x in ['수리서비스', '광학제품', '요식업소', '음료식품', '문화취미', '신변잡화', '레저업소', '자동차정비', '숙박', '레저용품', '의복'] else x) # '레저용품', ''

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

color_discrete_map = {'CAT1': 'rgb(85,107,47)', 'CAT2': 'rgb(169,169,169)', 'CAT3': 'rgb(250,128,114)'}
fig = px.scatter(scr_s, x="slope_middle", y="slope_after",hover_data=["cat"],text="catnm",template= 'plotly_white',color = 'group',color_discrete_map=color_discrete_map)
fig.update_traces(textfont_size=15)

fig.update_traces(textposition='bottom center', 
                  marker=dict(size=15))

fig.update_layout(
    showlegend=False,
    title=dict(text="회복도 및 타격도 / 서울 전체", font=dict(size=20)),
    xaxis_title=dict(text="타격도 (신천지 - 사회적 거리두기)", font=dict(size=16)),
    yaxis_title=dict(text="회복도 (사회적 거리두기 - 최근)", font=dict(size=16)),
    autosize=False,
    width=600,
    height=620,
    margin=dict(l=20,r=20,b=50,t=50,pad=4),
    shapes=[dict(type="circle",xref="x",yref="y",
                 x0=np.min(group1['slope_middle'])-0.3,y0=np.min(group1['slope_after'])-0.1,
                 x1=max(group1['slope_middle'])+0.62,y1=max(group1['slope_after'])+0.2,
                 opacity=0.2,fillcolor="pink",line_color="pink"),
            dict(type="circle",xref="x",yref="y",
                 x0=np.min(group2['slope_middle'])-0.16,y0=np.min(group2['slope_after'])-0.1,
                 x1=max(group2['slope_middle'])+0.05,y1=max(group2['slope_after'])+0.1,
                 opacity=0.2,fillcolor="lightgrey",line_color="lightgrey"),
            dict(type="circle",xref="x",yref="y",
                 x0=np.min(group3['slope_middle'])-0.17,y0=np.min(group3['slope_after'])-0.13,
                 x1=max(group3['slope_middle'])+0.05,y1=max(group3['slope_after'])+0.1,
                 opacity=0.1,fillcolor="green",line_color="green")])

fig.add_shape(dict(type="line", 
                   x0=0.5,y0=-2.2, x1=0.5,y1=2.2,
                   line=dict(color="red",width=1,dash="dash")))

fig.add_shape(dict(type="line", 
                   x0=-1.4,y0=0.5, x1=2,y1=0.5,
                   line=dict(color="red",width=1,dash="dash")))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=[scr_s.loc['SC_CNT_유통업','slope_middle']],
        y=[scr_s.loc['SC_CNT_유통업','slope_after']],
        marker=dict(
            color='crimson',
            size=20,
            line=dict(
                color='black',
                width=4
            )
        ),
        showlegend=False
    )
)

fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

color_discrete_map = {'CAT1': 'rgb(85,107,47)', 'CAT2': 'rgb(169,169,169)', 'CAT3': 'rgb(250,128,114)'}
fig = px.scatter(scr_s, x="slope_middle", y="slope_after",hover_data=["cat"],text="catnm",template= 'plotly_white',color = 'group',color_discrete_map=color_discrete_map)
fig.update_traces(textfont_size=15)

fig.update_traces(textposition='bottom center', 
                  marker=dict(size=15))

fig.update_layout(
    showlegend=False,
    title=dict(text="회복도 및 타격도 / 서울 전체", font=dict(size=20)),
    xaxis_title=dict(text="타격도 (신천지 - 사회적 거리두기)", font=dict(size=16)),
    yaxis_title=dict(text="회복도 (사회적 거리두기 - 최근)", font=dict(size=16)),
    autosize=False,
    width=600,
    height=800,
    margin=dict(l=20,r=20,b=50,t=50,pad=4),
    shapes=[dict(type="circle",xref="x",yref="y",
                 x0=np.min(group1['slope_middle'])-0.3,y0=np.min(group1['slope_after'])-0.1,
                 x1=max(group1['slope_middle'])+0.62,y1=max(group1['slope_after'])+0.2,
                 opacity=0.2,fillcolor="pink",line_color="pink"),
            dict(type="circle",xref="x",yref="y",
                 x0=np.min(group2['slope_middle'])-0.16,y0=np.min(group2['slope_after'])-0.1,
                 x1=max(group2['slope_middle'])+0.05,y1=max(group2['slope_after'])+0.1,
                 opacity=0.2,fillcolor="lightgrey",line_color="lightgrey"),
            dict(type="circle",xref="x",yref="y",
                 x0=np.min(group3['slope_middle'])-0.17,y0=np.min(group3['slope_after'])-0.13,
                 x1=max(group3['slope_middle'])+0.05,y1=max(group3['slope_after'])+0.1,
                 opacity=0.1,fillcolor="green",line_color="green")])

fig.add_shape(dict(type="line", 
                   x0=0.5,y0=-2.2, x1=0.5,y1=2.2,
                   line=dict(color="red",width=1,dash="dash")))

fig.add_shape(dict(type="line", 
                   x0=-1.4,y0=0.5, x1=2,y1=0.5,
                   line=dict(color="red",width=1,dash="dash")))


fig.show()

# **2.유통업 매출건수 증감률 시계열 그래프**

In [ ]:
fig = plt.figure(figsize=(16,6))

plt.plot(xx.index, xx['SC_CNT_유통업'], linestyle='-', marker='o', color='crimson')

mycolor ='grey'; min=-0.28
plt.axvline(x='02-18',color=mycolor,linestyle='--')
plt.text('02-18',min,'신천지', ha='center', va='center',rotation='horizontal', backgroundcolor='white', fontsize=15)
plt.axvline(x='03-22', color=mycolor,linestyle='--')
plt.text('03-22',min, '사회적 거리두기', ha='center', va='center',rotation='horizontal', backgroundcolor='white', fontsize=15)

plt.title('신한카드 유통업 매출건수 증감률 시계열 그래프', size=30)
plt.ylim(-0.3, -0.01)
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)